## Dynamical Analysis: Process DSF data - Step 0 (Possibly on the cluster)

We want to compute the `Dynamical Structure Factor`, which is given by

$$ S_{\vec{q}}(w) = \sum_{n=1}^{nev} \left| \left\langle \Psi_n^X \left| \hat{S}_{\vec{q}}^{\alpha} \right| \Psi_0 \right\rangle \right| ^2 \delta(\omega - (E_n^X - E_0)) $$

$$ S_{\vec{q}}(w) = \sum_{n=1}^{nev} \left| \left\langle \Psi_n^X \left| \hat{S}_{\vec{q}}^{\alpha} \right| \Psi_0 \right\rangle \right| ^2 \frac{1}{\pi} \frac{\epsilon^2}{(\omega - (E_n^X - E_0))^2 + \epsilon^2} $$

Where

$$ \hat{S}_{\vec{q}}^{\alpha} = \frac{1}{\sqrt{L}} \sum_{i = 0}^{L-1} e^{i \vec{q} \cdot \vec{r}_i} \hat{S}_i^{\alpha} $$

So, we can rewrite the DSF as:
$$ S_{\vec{q}}(w) = \sum_{n=1}^{nev} \left| \left\langle \Psi_n^X \left| \frac{1}{\sqrt{L}} \sum_{i=0}^{L-1} e^{i \vec{q} \cdot \vec{r}_i} \hat{S}_i^{\alpha} \right| \Psi_0 \right\rangle \right| ^2 \frac{1}{\pi} \frac{\epsilon^2}{(\omega - (E_n^X - E_0))^2 + \epsilon^2} $$

$$ S_{\vec{q}}(w) = \sum_{n=1}^{nev} \left| \frac{1}{\sqrt{L}} \sum_{i=0}^{L-1} e^{i \vec{q} \cdot \vec{r}_i} \left\langle \Psi_n^X \left| \hat{S}_i^{\alpha} \right| \Psi_0 \right\rangle \right| ^2 \frac{1}{\pi} \frac{\epsilon^2}{(\omega - (E_n^X - E_0))^2 + \epsilon^2} $$

From the code we obtain a file with two columns: 
1. the first one, has the values of $ \Delta_n = E_n^X -E_0 $
2. the second one, has the values of the coefficient $ \left\langle \Psi_n^X \left| \hat{S}_i^{\alpha} \right| \Psi_0 \right\rangle $

In [ ]:
import numpy as np
import cmath as m
import csv

### Parameters

In [ ]:
# Lattice

dim      = 2            # Dimension of the lattice
nspins   = 16           # Number of spins in the system    
Lx       = 4            # Number of spins in the system in x direction
Ly       = 4            # Number of spins in the system in y direction

r_idx    = [i for i in range(nspins)]
r_idxX   = [i//Ly for i in r_idx]
r_idxY   = [i%Ly for i in r_idx]

rep      = 10         # Disorder realizations
nev      = 200          # Number of excited states used

In [ ]:
# Reciprocal lattice

nQ       = nspins       # Number of q values to analyse

pi       = np.arccos(-1)
q_idx    = [i for i in range(nQ)]
q_idxX   = [(i//Ly)*2*pi/Lx for i in q_idx]
q_idxY   = [(i%Ly)*2*pi/Ly for i in q_idx]

q_X      = [(i//Ly) for i in q_idx]
q_Y      = [(i%Ly) for i in q_idx]

In [ ]:
# Hamiltonian parameters

J1           = 1.0      # Nearest neighbours coupling
D1           = 1.0      # Nearest neighbours anisotropy
J2           = 0.4      # Next-nearest neighbours coupling
D2           = 0.4      # Next-nearest neighbours anisotropy
dis_strength = 0.1      # Strength of the disorder

In [ ]:
# Frequency parametes

nW       = 5001         # Number of frequency values to plot
w_min    = 0            # Minimum value of frequency to plot
w_max    = 5            # Maximum value of frequency to plot
epsilon  = 1.0/nspins   # Constant to compute the Dirac delta function

In [ ]:
# Files

#pathINdata  = '/tmp/RUNS/dis' + str(dis_strength) + '/' + str(nspins) + 'nnn' + str(J2) + '/'
#pathOUTdata = '/tmp/RUNS/dis' + str(dis_strength) + '/' + str(nspins) + 'nnn' + str(J2) + '/'
pathINdata  = '/home/ale/RUNS/dis' + str(dis_strength) + '/' + str(nspins) + 'nnn' + str(J2) + '/'
pathOUTdata = '/home/ale/RUNS/dis' + str(dis_strength) + '/' + str(nspins) + 'nnn' + str(J2) + '/'

### 1. Read data from files

In [ ]:
# DeltaEn and factorN have the values of (En-E0) and the preceeding factor of the DSF
DeltaEn = np.empty((nev,rep))
factorNSPIN = np.empty((nev*nspins,rep))

# Loop over disorder realizations
for dis_rep in range(rep):
    
    # Loop over excited states
    with open(pathINdata + 'dsf_d' + str(dis_rep) + '_ns' + str(nspins) + '_nev' + str(nev) + '.dat') as INfile:
    #with open(path2INdata + 'dsf_NODISORDER' + '_ns' + str(nspins) + '_nev' + str(nev) + '.dat') as INfile:
        array = np.genfromtxt(INfile)
        tmp_DeltaEn, tmp_factorN = array.T
            
    
    DeltaEn[:,dis_rep] = tmp_DeltaEn[0:nev]
    factorNSPIN[:,dis_rep] = tmp_factorN

### 2. Sum over lattice sites

So, basically, one has to do the part:

$$ \frac{1}{\sqrt{L}} \sum_{i=0}^{L-1} e^{i \vec{q} \cdot \vec{r}_i} \left\langle \Psi_n^X \left| \hat{S}_i^{\alpha} \right| \Psi_0 \right\rangle $$

In [ ]:
# Funtion to compute the Dirac delta

e2     = epsilon*epsilon
pi     = np.arccos(-1)
inv_pi = 1.0/pi

def DiracDelta(w,deltaEn,e2):
    return inv_pi*e2/(e2 + (w-deltaEn)*(w-deltaEn))

In [ ]:
# Compute the exponential coefficient

expCoeff = np.empty((nQ,nspins),dtype=np.complex)

for q in range(nQ):
    for spin in range(nspins):
        coeff = q_idxX[q]*r_idxX[spin] + q_idxY[q]*r_idxY[spin]
        expCoeff[q,spin] = m.exp(1j*coeff)

In [ ]:
# Sum over the spins

factorN_cplx = np.zeros((nQ,rep,nev),dtype=np.complex)

for q in range(nQ):
    for n in range(nev):
        for i in range(nspins):
            offset = i * nev
            factorN_cplx[q,:,n] = factorN_cplx[q,:,n] + expCoeff[q,i]*factorNSPIN[n+offset,:]

factorN = np.empty((nQ,rep,nev))
factorN[:,:,:] = abs(factorN_cplx[:,:,:])**2/nspins

### 3. Sum over eigenstates

In [ ]:
# Sum over the eigenstates (multiplying by the Dirac delta)

tmp_Sw = np.zeros((rep,nQ,nW))
w  = np.linspace(w_min,w_max,nW)

for dis_rep in range(rep):

    # Compute the values of DeltaDirac to make the next step faster
    deltad = np.zeros((nW,nev))
    for n in range(nev):
        for wi in range(nW):
            deltad[wi,n] = DiracDelta(w[wi],DeltaEn[n,dis_rep],e2)

    for q in range(nQ):                
        for wi in range(nW):
            mid_array = factorN[q,dis_rep,:]*deltad[wi,:]
            tmp_Sw[dis_rep,q,wi] = np.sum(mid_array)

# Save the values of Sq(w) for all the disorder realizations
for dis_rep in range(rep):
    header = "disorder realization:" + str(dis_rep) + "\n"+"        w                 Sw"
    np.savetxt(pathOUTdata + '/Sq/Sq_' + str(dis_rep) + '.csv', np.c_[w,tmp_Sw[dis_rep,:,:].T], header=header, fmt='%16.13e', delimiter=' ',)

### 4. Average over disorder realizations

In [ ]:
# Compute average over disorder realizations

Sw = np.zeros((nQ,nW))

Sw = np.sum(tmp_Sw, axis=0)
Sw *= 2*pi
Sw /= rep
Sw = Sw.T
    
# Save the values the dynamical structure factor
header = "        w                 Sw"
np.savetxt(pathOUTdata + 'Sq.csv', np.c_[w,Sw], header=header, fmt='%16.13e', delimiter=' ',)      